# **BERT-QA REGLAMENTO UPB**

Entregable #3 de la materia **Analítica de Datos No Estructurados / Minería Multimedia**.

El objetivo es hacer el fine-tunning de un modelo **BERT en español** para responder preguntas sobre el reglamento estudiantil de la UPB a partir de un dataset .json.

> Simón Correa Marín

### **Librerías y configuración**

In [1]:
import os
import json
import requests
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    pipeline,
)

In [2]:
import torch

print("Versión de PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("MPS disponible:", torch.backends.mps.is_available())

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Usando MPS (GPU Apple Silicon)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Usando GPU CUDA")
else:
    device = torch.device("cpu")
    print("Usando CPU")

device

Versión de PyTorch: 2.9.1
CUDA disponible: False
MPS disponible: True
Usando MPS (GPU Apple Silicon)


device(type='mps')

### **Carga de datos desde la URL - Reglamento UPB**

In [3]:
url = "https://robertohincapie.com/data/faq_reglamento_upb_completo_final.json"

response = requests.get(url)
response.raise_for_status()  # lanza error si algo falla

data = response.json()

print("Número de questions en el dataset:", len(data))

# Dataframe
df = pd.DataFrame(data)
df.head()

Número de questions en el dataset: 57


,context,question,answer,item_id,answer_start,answer_end
0,Para ingresar a la Universidad Pontificia Boli...,¿Qué se necesita para ingresar a un programa d...,diligenciar y pagar el formulario de inscripción,7b2e6743-75ea-472e-9330-f87a8942d28b,73,121
1,El ingreso a la Universidad puede hacerse como...,¿Cuáles son las formas de ingreso a la UPB?,"como estudiante nuevo, por transferencia inter...",9ccbd0aa-aa32-49d1-b15c-adec3b7b7333,42,133
2,Los estudiantes extranjeros deben cumplir con ...,¿Qué deben hacer los aspirantes extranjeros pa...,cumplir con los requisitos migratorios exigido...,1aedf782-75f4-4ab3-93cc-fa66235f9be3,34,103
3,Un estudiante nuevo es aquel que es aceptado p...,¿Quién se considera estudiante nuevo en la UPB?,aquel que es aceptado por primera vez en un pr...,857e420b-ae46-49a0-b811-c9121cb91c58,23,112
4,El reintegro es el proceso mediante el cual un...,¿Qué es el reintegro en la UPB?,el proceso mediante el cual un estudiante que ...,ff7f430f-319e-4ba9-9464-ae5efc54b317,16,103


### **Exploración del dataset**

In [4]:
print("Columnas del dataset:", df.columns.tolist())
print(df.info())

Columnas del dataset: ['context', 'question', 'answer', 'item_id', 'answer_start', 'answer_end']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   context       57 non-null     object
 1   question      57 non-null     object
 2   answer        57 non-null     object
 3   item_id       57 non-null     object
 4   answer_start  57 non-null     int64 
 5   answer_end    57 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.8+ KB
None


### **División de los datos**

In [5]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

print("Train:", len(train_df))
print("Valid:", len(valid_df))

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

raw_datasets = DatasetDict({
    "train": train_ds,
    "validation": valid_ds,
})

raw_datasets

Train: 45
Valid: 12


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'item_id', 'answer_start', 'answer_end'],
        num_rows: 45
    })
    validation: Dataset({
        features: ['context', 'question', 'answer', 'item_id', 'answer_start', 'answer_end'],
        num_rows: 12
    })
})

### **Columna Answers**

In [6]:
def columna_answers(example):
    answer_text = example["answer"]
    start_char = example["answer_start"]
    return {
        "id": str(example["item_id"]),
        "context": example["context"],
        "question": example["question"],
        "answers": {
            "text": [answer_text],
            "answer_start": [start_char],
        },
    }

df_final = raw_datasets.map(columna_answers)

# Columnas necesarias
columnas = ["id", "context", "question", "answers"]
df_final = df_final.remove_columns(
    [c for c in df_final["train"].column_names if c not in columnas]
)

df_final["train"][0]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

{'context': 'Para obtener el título académico el estudiante debe culminar el plan de estudios, demostrar competencia en segunda lengua, y estar a paz y salvo académica, disciplinaria y financieramente. El título se entrega en ceremonia oficial y puede ser revocado si se demuestra que fue obtenido mediante fraude o falsedad.',
 'question': '¿Qué tipo de paz y salvo debe tener?',
 'id': 'e8d741df-5cb2-4c71-b9f4-c58b1059d1c6',
 'answers': {'answer_start': [145],
  'text': ['académica, disciplinaria y financieramente']}}

## **BERT en Español**

### **Tokenizer**

In [7]:
# Carga del tokenizer BERT en español

model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print("Tokenizer cargado desde:", model_checkpoint)

Tokenizer cargado desde: dccuchile/bert-base-spanish-wwm-cased


In [8]:
# Preparación de features para entrenamiento

max_length = 384   # Longitud máxima de secuencia
doc_stride = 128   # Solapamiento entre los chunks de contexto

def prepare_train_features(examples):
    # Limpiamos espacios extra en las preguntas
    examples["question"] = [q.strip() for q in examples["question"]]
    
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",          # Truncar únicamente el contexto
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")
    
    start_positions = []
    end_positions = []
    
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        sample_idx = sample_mapping[i]
        answers = examples["answers"][sample_idx]
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])
        
        sequence_ids = tokenized.sequence_ids(i)
        
        # Índices del contexto dentro de la secuencia de tokens
        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        
        idx = len(sequence_ids) - 1
        while idx >= 0 and sequence_ids[idx] != 1:
            idx -= 1
        context_end = idx
        
        # Si la respuesta no cae completamente en este chunk, apuntamos al token CLS
        if not (offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            # Encontrar token de inicio
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_token_index = idx - 1
            
            # Encontrar token de fin
            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_token_index = idx + 1
            
            start_positions.append(start_token_index)
            end_positions.append(end_token_index)
    
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

tokenized_train = df_final["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=df_final["train"].column_names,
)

tokenized_valid = df_final["validation"].map(
    prepare_train_features,
    batched=True,
    remove_columns=df_final["validation"].column_names,
)

tokenized_train

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 45
})

### **Modelo BERT para QA**

In [9]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model.to(device)

model

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

### **Configuración del Trainer**

In [10]:
batch_size = 4 # Se puede bajar a 2

training_args = TrainingArguments(
    output_dir="qa-bert-upb-checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
)

/var/folders/x2/wzyt5ll12z77w4qw_gm51n8w0000gn/T/ipykernel_71106/2393597552.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### **Entrenamiento**

In [11]:
train_result = trainer.train()
train_result

/Users/simon/Documents/bert-qa-upb/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,3.684337
2,No log,3.243272
3,No log,3.237133


/Users/simon/Documents/bert-qa-upb/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/simon/Documents/bert-qa-upb/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=36, training_loss=3.4522357516818576, metrics={'train_runtime': 2243.0137, 'train_samples_per_second': 0.06, 'train_steps_per_second': 0.016, 'total_flos': 26456296619520.0, 'train_loss': 3.4522357516818576, 'epoch': 3.0})

### **Modelo y tokenizer luego del fine-tunning**

In [12]:
save_dir = "qa-bert-upb-model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Modelo guardado en: {os.path.abspath(save_dir)}")

Modelo guardado en: /Users/simon/Documents/bert-qa-upb/qa-bert-upb-model


## **Inferencia - QA**

In [13]:
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer
)

qa_pipeline

Device set to use mps:0


### **Prueba sobre validación**

In [14]:
for i in range(5):
    ejemplo = df_final["validation"][i]
    context = ejemplo["context"]
    question = ejemplo["question"]
    gold_answer = ejemplo["answers"]["text"][0]
    
    pred = qa_pipeline({
        "question": question,
        "context": context
    })
    
    print(f"\n===== Ejemplo {i+1} =====")
    print("Pregunta:       ", question)
    print("Respuesta real: ", gold_answer)
    print("Respuesta modelo:", pred['answer'])
    print("Score:", round(pred['score'], 3))

/Users/simon/Documents/bert-qa-upb/.venv/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



===== Ejemplo 1 =====
Pregunta:        ¿Qué se necesita para ingresar a un programa de pregrado en la UPB?
Respuesta real:  diligenciar y pagar el formulario de inscripción
Respuesta modelo: cumplir con los requisitos de admisión establecidos por la unidad académica
Score: 0.011

===== Ejemplo 2 =====
Pregunta:        ¿Qué debe hacer un estudiante al reintegrarse respecto al plan de estudios?
Respuesta real:  acogerse al plan de estudios y tarifas vigentes al momento del retorno
Respuesta modelo: acogerse al plan de estudios
Score: 0.055

===== Ejemplo 3 =====
Pregunta:        ¿Cuántas veces puede reservar cupo?
Respuesta real:  hasta por dos períodos
Respuesta modelo: reserva de cupo
Score: 0.026

===== Ejemplo 4 =====
Pregunta:        ¿Cuál es la nota mínima para aprobar un curso?
Respuesta real:  una nota igual o superior a 3.00
Respuesta modelo: nota igual o superior a 3.00
Score: 0.038

===== Ejemplo 5 =====
Pregunta:        ¿Qué tipos de evaluación existen?
Respuesta real:  form

### **Prueba manual - Input Usuario**

In [15]:
# Tomamos un contexto representativo (por simplicidad, uno de los ejemplos)
ejemplo_contexto = df_final["train"][0]["context"]

pregunta_usuario = "¿Cuál es la nota mínima para aprobar un curso en la UPB?"

pred = qa_pipeline({
    "question": pregunta_usuario,
    "context": ejemplo_contexto
})

print("Pregunta del usuario:", pregunta_usuario)
print("Respuesta del modelo:", pred["answer"])
print("Score:", round(pred["score"], 3))

Pregunta del usuario: ¿Cuál es la nota mínima para aprobar un curso en la UPB?
Respuesta del modelo: demostrar competencia en segunda lengua
Score: 0.056


Se debe tener en cuenta que el modelo no conoce todo el reglamento UPB, solo puede extraer una frase del contexto que se le pase, si el contexto no contiene la respuesta, el modelo de igual manera debe escoger alguna respuesta (span de texto) aun así no tenga sentido. Lo que sucedió en este caso es que se le dió un contexto equivocado al modelo. Para evitar esto, se debe buscar un contexto relevante en el dataset antes de llamar al modelo.

### **Prueba con contexto buscado**

In [29]:
# Contextos que hablen de "nota mínima" o "nota igual o superior a 3.00"
mask = df["context"].str.contains("nota mínima", case=False, na=False) | \
       df["context"].str.contains("curso", case=False, na=False)

df[mask].head()

,context,question,answer,item_id,answer_start,answer_end
12,Las evaluaciones pueden ser formativas o acumu...,¿Qué tipo de evaluación no requiere nota oblig...,evaluación formativa,66daf9e0-1b2f-4fad-adf1-eefdc56eb90e,58,78
13,Las calificaciones en la UPB se expresan en es...,¿Cuál es la nota mínima para aprobar un curso?,una nota igual o superior a 3.00,76c690e3-8ce1-482e-988b-04ead6ea1115,110,142
34,Las evaluaciones se clasifican en formativas y...,¿Qué tipos de evaluación existen?,formativas y acumulativas,6bf0d797-c0eb-4f32-a1c1-019e7e4e1f53,34,59
35,Las evaluaciones se clasifican en formativas y...,¿Cuál es la nota mínima para aprobar?,3.00,ae7bc4cd-7345-44ad-9919-120b82ddd263,104,108
36,Las evaluaciones se clasifican en formativas y...,¿Qué escala se usa para calificar?,escala de 0.00 a 5.00,d3538c8e-5e81-4015-b9e6-61c67d78e325,143,164


In [31]:
contexto_nota = df[mask].iloc[1]["context"]

pregunta_usuario = "¿Cuál es la nota mínima para aprobar un curso en la UPB?"

pred = qa_pipeline({
    "question": pregunta_usuario,
    "context": contexto_nota
})

print("Pregunta del usuario:", pregunta_usuario)
print("Respuesta del modelo:", pred["answer"])
print("Score:", round(pred["score"], 3))

/Users/simon/Documents/bert-qa-upb/.venv/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Pregunta del usuario: ¿Cuál es la nota mínima para aprobar un curso en la UPB?
Respuesta del modelo: nota igual o superior a 3.00
Score: 0.037


En un sistema real, este modelo se integraría con un módulo de recuperación de documentos (IR), encargándose BERT únicamente de extraer la respuesta del fragmento más relevante.